In [ ]:
__author__ = "Weiyi, Zhang" 
__version__ = "1.0" 

In [2]:
import numpy as np
import pandas as pd
import sys, os
import gc
import math

cutoff_r = 6
cutoff_theta = 6
###This code only works while 2*max(cutoffs) <= min(periods)

###Angle calc are weighted by exp() function about distance of atoms as default

with open('min10051','r') as f:
    lines = f.readlines()

atoms = [line.split() for line in lines[26:]]
lines.clear()
df = pd.DataFrame(atoms)
df.columns = ["idx", "type", "x", "y","z"]

df['idx'] = [int(e) for e in df['idx']]
df['type'] = [int(e) for e in df['type']]
df['x'] = [float(e) for e in df['x']]
df['y'] = [float(e) for e in df['y']]
df['z'] = [float(e) for e in df['z']]
positions = df[['x','y','z']].to_numpy()


box = ['3.1781394890167824e+00 4.9274350212712889e+01 xlo xhi\n',
 '3.1781394890167824e+00 4.9274350212712889e+01 ylo yhi\n',
 '3.1781394890167824e+00 4.9274350212712889e+01 zlo zhi\n']
for i in range(3):
    box[i] = [float(e) for e in box[i].split()[:2]]
box = np.array(box)
box = box[:,1]-box[:,0]

def distance(p1,p2=[0,0,0]):
    t = np.array(p1)-np.array(p2)
    return np.dot(t,t)**0.5

In [2]:
v_matrix = np.zeros((len(positions),len(positions),len(positions[0])))
d_matrix = np.zeros((len(positions),len(positions)))
d2_matrix = np.zeros((len(positions),len(positions)))

In [3]:
for i in range(len(positions)):
    v_matrix[i,:,:] = positions[:] - positions[i]
v_matrix = np.absolute(v_matrix)
v_matrix = np.minimum(box - v_matrix, v_matrix)
v_matrix

array([[[ 0.        ,  0.        ,  0.        ],
        [ 5.55971493,  5.40843398,  7.51967583],
        [ 7.27689891,  0.45229072,  6.33110402],
        ...,
        [10.38777393,  4.26228294, 12.46817387],
        [20.5692068 ,  2.17167227,  3.04268404],
        [16.93482832,  2.41798734,  3.26979981]],

       [[ 5.55971493,  5.40843398,  7.51967583],
        [ 0.        ,  0.        ,  0.        ],
        [12.83661384,  4.95614326,  1.18857181],
        ...,
        [15.94748886,  9.67071692, 19.9878497 ],
        [19.96728899,  7.58010625, 10.56235987],
        [11.37511339,  7.82642132, 10.78947564]],

       [[ 7.27689891,  0.45229072,  6.33110402],
        [12.83661384,  4.95614326,  1.18857181],
        [ 0.        ,  0.        ,  0.        ],
        ...,
        [ 3.11087502,  4.71457366, 18.79927789],
        [13.29230789,  2.62396299,  9.37378806],
        [21.88448349,  2.87027806,  9.60090383]],

       ...,

       [[10.38777393,  4.26228294, 12.46817387],
        [15

In [4]:
for i in range(len(positions)):
    for j in range(i):
        d2_matrix[i,j] = d2_matrix[j,i] = np.dot(v_matrix[i,j,:],v_matrix[i,j,:])
        d_matrix[i,j] = d_matrix[j,i] = d2_matrix[i,j]**0.5
    if i%500 == 0:
        print(i)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500


In [ ]:
np.save('vectors.npy',v_matrix)
np.save('d_matrix.npy',d_matrix)
np.save('d2_matrix.npy',d2_matrix)

In [ ]:
d_matrix.shape, d_matrix, d2_matrix.shape, d2_matrix

In [5]:
type1_idx,type2_idx,type3_idx,type4_idx = np.array(df[df["type"] == 1]['idx'])-1,np.array(df[df["type"] == 2]['idx'])-1,np.array(df[df["type"] == 3]['idx'])-1,np.array(df[df["type"] == 4]['idx'])-1
type_idxs = [type1_idx,type2_idx,type3_idx,type4_idx]

In [14]:
set(df["type"])

{1, 2, 3, 4}

In [6]:
typN=4
MuR=np.linspace(1,6,10)
MuS=np.linspace(0,np.pi,10)
SigmaR=0.3
SigmaS=0.3
import math
def gaussian(x, mu, sig = SigmaS):
    return np.exp( - ( (np.array(x) - mu) / sig)**2 / 2) / (sig*(2*math.pi)**0.5)
AEV = np.zeros((len(positions),typN,typN,len(MuR),len(MuS)+1))

In [7]:
for atom_idx in range(len(positions)):
    for type1 in range(typN):
        for idx, r in enumerate(MuR):
            d_type1 = [d for d in d_matrix[atom_idx,type_idxs[type1]] if d <= r and d != 0.0]
            AEV[atom_idx, type1, :, idx, 0] = sum(gaussian(d_type1, r, sig = SigmaR))
    if atom_idx%500 == 0:
        print(atom_idx)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500


In [10]:
AEV[:,:,0,:,0],AEV[0,:,0,:,0]

(array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          3.80554228e+00, 4.86258847e+00, 2.71827147e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          6.44969507e+00, 9.34874577e+00, 4.94970450e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          1.65204527e+00, 2.80426391e+00, 2.50226486e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 1.25209656e+00, 1.18539836e-01]],
 
        [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          1.30707288e+00, 2.88863724e+00, 3.24585885e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          1.18879606e+01, 8.84204252e+00, 3.92363356e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          2.35195698e+00, 8.52234342e-01, 8.17163778e-03],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 2.47298128e-01, 1.49063126e-03]],
 
        [[0.000000

In [13]:
import time
for atom_idx in range(len(positions)):
    for type1 in range(typN):
        for type2 in range(typN):
            angles = []
            pres_r = 0
            pres_d_type1 = []
            pres_d_type2 = []
            for idx_r, r in enumerate(MuR):
                d_type1 = [[i,d2,d] for i, d2, d in zip(type_idxs[type1],d2_matrix[atom_idx,type_idxs[type1]],d_matrix[atom_idx,type_idxs[type1]]) if pres_r < d <= r and d != 0.0]
                d_type2 = [[i,d2,d] for i, d2, d in zip(type_idxs[type1],d2_matrix[atom_idx,type_idxs[type1]],d_matrix[atom_idx,type_idxs[type1]]) if pres_r < d <= r and d != 0.0]
                for atom1_idx, atom1_d2, atom1_d in d_type1:
                    for atom2_idx, atom2_d2, atom2_d in d_type2:
                        d_12 = d2_matrix[atom2_idx,atom1_idx]
                        #print((-d_12+atom1_d2+atom2_d2)/2.0/atom1_d/atom2_d)
                        theta = np.arccos((-d_12+atom1_d2+atom2_d2)/2.0/atom1_d/atom2_d)
                        angles.append(theta)
                for atom1_idx, atom1_d2, atom1_d in pres_d_type1:
                    for atom2_idx, atom2_d2, atom2_d in d_type2:
                        d_12 = d2_matrix[atom2_idx,atom1_idx]
                        #print((-d_12+atom1_d2+atom2_d2)/2.0/atom1_d/atom2_d)
                        theta = np.arccos((-d_12+atom1_d2+atom2_d2)/2.0/atom1_d/atom2_d)
                        angles.append(theta)
                for atom1_idx, atom1_d2, atom1_d in d_type1:
                    for atom2_idx, atom2_d2, atom2_d in pres_d_type2:
                        d_12 = d2_matrix[atom2_idx,atom1_idx]
                        #print((-d_12+atom1_d2+atom2_d2)/2.0/atom1_d/atom2_d)
                        theta = np.arccos((-d_12+atom1_d2+atom2_d2)/2.0/atom1_d/atom2_d)
                        angles.append(theta)
                pres_d_type1 = d_type1
                pres_d_type2 = d_type2
                pres_r = r
                for idx_s, l in enumerate(MuR):
                    AEV[atom_idx, type1, type2, idx_r, idx_s+1] = sum(gaussian([angle for angle in angles if angle < l], l, sig = SigmaS))
    
    if atom_idx%50 == 0:
        print(atom_idx,time.time())

/bigdata/greaneylab/wzhan097/.conda/envs/ML_cpu/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in arccos
  app.launch_new_instance()


0 1581381617.9393182
50 1581381643.233722
100 1581381668.8218946
150 1581381694.1889958
200 1581381719.6752968
250 1581381745.2874177
300 1581381770.886327
350 1581381796.5085862
400 1581381822.0829616
450 1581381847.6108203
500 1581381873.3939133
550 1581381899.07999
600 1581381924.6360695
650 1581381950.1807497
700 1581381975.903687
750 1581382001.542601
800 1581382027.483959
850 1581382053.1283395
900 1581382078.566926
950 1581382103.979829
1000 1581382129.595432
1050 1581382155.326986
1100 1581382180.8160405
1150 1581382206.5914056
1200 1581382232.099827
1250 1581382257.5167065
1300 1581382283.06847
1350 1581382308.4600568
1400 1581382333.8970227
1450 1581382359.2846746
1500 1581382384.71559
1550 1581382410.1681895
1600 1581382435.7396023


KeyboardInterrupt: 

In [12]:
AEV[:,:,0,:,0],AEV[0,:,0,:,0]

(array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          3.80554228e+00, 4.86258847e+00, 2.71827147e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          6.44969507e+00, 9.34874577e+00, 4.94970450e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          1.65204527e+00, 2.80426391e+00, 2.50226486e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 1.25209656e+00, 1.18539836e-01]],
 
        [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          1.30707288e+00, 2.88863724e+00, 3.24585885e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          1.18879606e+01, 8.84204252e+00, 3.92363356e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          2.35195698e+00, 8.52234342e-01, 8.17163778e-03],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 2.47298128e-01, 1.49063126e-03]],
 
        [[0.000000

In [15]:
time.ctime(time.time())

'Mon Feb 10 17:12:10 2020'